
### Requisitos para os arquivos da pasta partidas

Objetivo: realizar processo de ETL e disponibilizar os dados em camada adequada.

1. Ler o arquivos csv da pasta no Datalake [ok]
2. Renomear colunas para português usando snake case [ok]
3. Definir a tipagem de colunas correta [ok]
4. Coluna com data de cosumo do arquivo [ok]
5. Salvar dados em formato parquet na camada adequada [ok]

In [1]:
%run ../../Configuração-e-Utilitários/Variaveis.ipynb

In [2]:
%run ../../Configuração-e-Utilitários/Funções.ipynb

### Leitura do arquivo e definição de schema

In [5]:
# Lendo o arquivo passando o schema como parâmetro
jogadores_df = spark.read.json(f"{caminho_bronze}/jogadores.json")

jogadores_df.show()

+---+--------------------+-----------------+------+------+-------+--------------------+-----------------+-----------+--------------------+------+
|age|               birth|        firstname|height|    id|injured|            lastname|             name|nationality|               photo|weight|
+---+--------------------+-----------------+------+------+-------+--------------------+-----------------+-----------+--------------------+------+
| 24|{Brazil, 1999-05-...|         Vinícius|  NULL| 10367|  false|      Lopes da Silva|   Vinícius Lopes|     Brazil|https://media-4.a...|  NULL|
| 20|{Brazil, 2003-04-...|        Ryan Luka|185 cm|266020|  false|   Cordeiro de Souza|        Ryan Luka|     Brazil|https://media-4.a...| 76 kg|
| 24|{Brazil, 1999-05-...|Vinicius Balieiro|  NULL|278496|  false|Lourenco De Carvalho|Vinicius Balieiro|     Brazil|https://media-4.a...|  NULL|
| 22|{Brazil, 2001-08-...|             Yago|174 cm|305815|  false|   de Paula Ferreira|    Yago Ferreira|     Brazil|https:/

In [6]:
jogadores_df.printSchema()

root
 |-- age: long (nullable = true)
 |-- birth: struct (nullable = true)
 |    |-- country: string (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- place: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- height: string (nullable = true)
 |-- id: long (nullable = true)
 |-- injured: boolean (nullable = true)
 |-- lastname: string (nullable = true)
 |-- name: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- photo: string (nullable = true)
 |-- weight: string (nullable = true)



### Lidando com a coluna aninhada 'birth'

In [7]:
from pyspark.sql.functions import explode, array

jogadores_df = jogadores_df.withColumn('country', explode(array(jogadores_df.birth.country))) \
                           .withColumn('date', explode(array(jogadores_df.birth.date))) \
                           .withColumn('place', explode(array(jogadores_df.birth.place)))


# Deletando a coluna aninhada
jogadores_df = jogadores_df.drop('birth')

# Imprimindo o df
jogadores_df.show()

+---+-----------------+------+------+-------+--------------------+-----------------+-----------+--------------------+------+--------+----------+------------+
|age|        firstname|height|    id|injured|            lastname|             name|nationality|               photo|weight| country|      date|       place|
+---+-----------------+------+------+-------+--------------------+-----------------+-----------+--------------------+------+--------+----------+------------+
| 24|         Vinícius|  NULL| 10367|  false|      Lopes da Silva|   Vinícius Lopes|     Brazil|https://media-4.a...|  NULL|  Brazil|1999-05-07|        NULL|
| 20|        Ryan Luka|185 cm|266020|  false|   Cordeiro de Souza|        Ryan Luka|     Brazil|https://media-4.a...| 76 kg|  Brazil|2003-04-30|        NULL|
| 24|Vinicius Balieiro|  NULL|278496|  false|Lourenco De Carvalho|Vinicius Balieiro|     Brazil|https://media-4.a...|  NULL|  Brazil|1999-05-28|        NULL|
| 22|             Yago|174 cm|305815|  false|   de P

### Tratando colunas 'height' e 'weight' e alterando booleanos para 'Sim'/'Não'

In [8]:
from pyspark.sql.functions import col, regexp_replace, lower

# Retirando os caracteres especificados
jogadores_df = jogadores_df.withColumn('height', regexp_replace(lower(col('height')), ' cm', '')) \
                           .withColumn('weight', regexp_replace(lower(col('weight')), ' kg', ''))

jogadores_df.show()

+---+-----------------+------+------+-------+--------------------+-----------------+-----------+--------------------+------+--------+----------+------------+
|age|        firstname|height|    id|injured|            lastname|             name|nationality|               photo|weight| country|      date|       place|
+---+-----------------+------+------+-------+--------------------+-----------------+-----------+--------------------+------+--------+----------+------------+
| 24|         Vinícius|  NULL| 10367|  false|      Lopes da Silva|   Vinícius Lopes|     Brazil|https://media-4.a...|  NULL|  Brazil|1999-05-07|        NULL|
| 20|        Ryan Luka|   185|266020|  false|   Cordeiro de Souza|        Ryan Luka|     Brazil|https://media-4.a...|    76|  Brazil|2003-04-30|        NULL|
| 24|Vinicius Balieiro|  NULL|278496|  false|Lourenco De Carvalho|Vinicius Balieiro|     Brazil|https://media-4.a...|  NULL|  Brazil|1999-05-28|        NULL|
| 22|             Yago|   174|305815|  false|   de P

### Ajustando o schema dos dados

In [9]:
jogadores_df.printSchema()

root
 |-- age: long (nullable = true)
 |-- firstname: string (nullable = true)
 |-- height: string (nullable = true)
 |-- id: long (nullable = true)
 |-- injured: boolean (nullable = true)
 |-- lastname: string (nullable = true)
 |-- name: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- photo: string (nullable = true)
 |-- weight: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date: string (nullable = true)
 |-- place: string (nullable = true)



In [12]:
from pyspark.sql.functions import col

jogadores_df = jogadores_df.withColumn('age', col('age').cast('integer')) \
                           .withColumn('height', col('height').cast('integer')) \
                           .withColumn('weight', col('weight').cast('integer')) \
                           .withColumn('id', col('id').cast('integer')) \
                           .withColumn('date', col('date').cast('date')) \

jogadores_df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- firstname: string (nullable = true)
 |-- height: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- injured: boolean (nullable = true)
 |-- lastname: string (nullable = true)
 |-- name: string (nullable = true)
 |-- nationality: string (nullable = true)
 |-- photo: string (nullable = true)
 |-- weight: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- date: date (nullable = true)
 |-- place: string (nullable = true)



### Alterando valores booleanos da coluna 'injured' para 'Sim'/'Não' 

In [13]:
from pyspark.sql.functions import when

jogadores_df = jogadores_df.withColumn('injured', col('injured').cast('string'))
jogadores_df = jogadores_df.withColumn('injured', when(col('injured') == 'true', 'Sim').otherwise('Não'))

jogadores_df.show()

+---+-----------------+------+------+-------+--------------------+-----------------+-----------+--------------------+------+--------+----------+------------+
|age|        firstname|height|    id|injured|            lastname|             name|nationality|               photo|weight| country|      date|       place|
+---+-----------------+------+------+-------+--------------------+-----------------+-----------+--------------------+------+--------+----------+------------+
| 24|         Vinícius|  NULL| 10367|    Não|      Lopes da Silva|   Vinícius Lopes|     Brazil|https://media-4.a...|  NULL|  Brazil|1999-05-07|        NULL|
| 20|        Ryan Luka|   185|266020|    Não|   Cordeiro de Souza|        Ryan Luka|     Brazil|https://media-4.a...|    76|  Brazil|2003-04-30|        NULL|
| 24|Vinicius Balieiro|  NULL|278496|    Não|Lourenco De Carvalho|Vinicius Balieiro|     Brazil|https://media-4.a...|  NULL|  Brazil|1999-05-28|        NULL|
| 22|             Yago|   174|305815|    Não|   de P

### Renomeando os atributos para português com uso do snake_case

In [14]:
jogadores_df = jogadores_df.withColumnRenamed('id', 'id_jogador') \
                           .withColumnRenamed('age', 'idade') \
                           .withColumnRenamed('firstname', 'primeiro_nome') \
                           .withColumnRenamed('lastname', 'ultimo_nome') \
                           .withColumnRenamed('name', 'nome_completo') \
                           .withColumnRenamed('nationality', 'nacionalidade') \
                           .withColumnRenamed('injured', 'lesionado?') \
                           .withColumnRenamed('country', 'pais_de_origem') \
                           .withColumnRenamed('date', 'data_de_nascimento') \
                           .withColumnRenamed('place', 'cidade_de_origem') \
                           .withColumnRenamed('height', 'altura (cm)') \
                           .withColumnRenamed('weight', 'peso (kg)') \
                           .withColumnRenamed('photo', 'link_imagem')

jogadores_df.show()

+-----+-----------------+-----------+----------+----------+--------------------+-----------------+-------------+--------------------+---------+--------------+------------------+----------------+
|idade|    primeiro_nome|altura (cm)|id_jogador|lesionado?|         ultimo_nome|    nome_completo|nacionalidade|         link_imagem|peso (kg)|pais_de_origem|data_de_nascimento|cidade_de_origem|
+-----+-----------------+-----------+----------+----------+--------------------+-----------------+-------------+--------------------+---------+--------------+------------------+----------------+
|   24|         Vinícius|       NULL|     10367|       Não|      Lopes da Silva|   Vinícius Lopes|       Brazil|https://media-4.a...|     NULL|        Brazil|        1999-05-07|            NULL|
|   20|        Ryan Luka|        185|    266020|       Não|   Cordeiro de Souza|        Ryan Luka|       Brazil|https://media-4.a...|       76|        Brazil|        2003-04-30|            NULL|
|   24|Vinicius Balieiro|

### Criando coluna com data de consumo

In [15]:
jogadores_df = adicionar_data_de_ingestao(
    df = jogadores_df,
    nome_coluna = 'data_de_ingestao',
    add_tempo = -3,
    unidade_de_medida = 'horas'
)

jogadores_df.show()

+-----+-----------------+-----------+----------+----------+--------------------+-----------------+-------------+--------------------+---------+--------------+------------------+----------------+----------------+
|idade|    primeiro_nome|altura (cm)|id_jogador|lesionado?|         ultimo_nome|    nome_completo|nacionalidade|         link_imagem|peso (kg)|pais_de_origem|data_de_nascimento|cidade_de_origem|data_de_ingestao|
+-----+-----------------+-----------+----------+----------+--------------------+-----------------+-------------+--------------------+---------+--------------+------------------+----------------+----------------+
|   24|         Vinícius|       NULL|     10367|       Não|      Lopes da Silva|   Vinícius Lopes|       Brazil|https://media-4.a...|     NULL|        Brazil|        1999-05-07|            NULL|      2024-05-01|
|   20|        Ryan Luka|        185|    266020|       Não|   Cordeiro de Souza|        Ryan Luka|       Brazil|https://media-4.a...|       76|        B

### Salvando arquivo

In [17]:
jogadores_df.write.mode('overwrite').parquet(f"{caminho_silver}/jogadores")

In [18]:
dbutils.notebook.exit('O arquivo "jogadores.json" foi processado e carregado na camada silver.')